In [ ]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
from pathlib import Path
from PIL import Image
#import cv2
#import dask.array as da    
import time
import keras
import tensorflow_addons as tfa

In [ ]:
data_path_df = pd.read_csv("final_data_path.csv")
data_path_df.head()

In [ ]:
data_path_df['faces'][0][0:19] + '_data' +  data_path_df['faces'][0][19:] + '/'

In [ ]:
lms_data = {}
faces_data = {}

for i in range (0, len(data_path_df)):
    count = 0
    count1 = 0
    basepath = Path('lms_data' +  data_path_df['faces'][i][19:] + '/')
    basepath1 = Path('faces_data' +  data_path_df['faces'][i][19:] + '/')
    files_in_basepath = basepath.iterdir()
    for item in files_in_basepath:
        count = count + 1
    count1 = int(np.round(0.9*count, decimals = 0))
    files_in_basepath = basepath.iterdir()
    for item in files_in_basepath:            
        if (count1 > 0):
            item = str(item)
            lms_data.setdefault('train', []).append(item)
            count1 = count1 - 1
            files_in_basepath1 = basepath1.iterdir()    
            for item1 in files_in_basepath1:
                item1 = str(item1)
                faces_data[item] = item1
                break
        else:
            item = str(item)
            lms_data.setdefault('test', []).append(item)
            files_in_basepath1 = basepath1.iterdir()    
            for item1 in files_in_basepath1:
                item1 = str(item1)
                faces_data[item] = item1
                break

In [ ]:
final_data = list(faces_data.keys())
random.shuffle(final_data)
train_data = final_data[0:int(np.round((0.8 * len(final_data)), decimals = 0))]
test_data = final_data[int(np.round((0.8 * len(final_data)), decimals = 0)):]
print(np.shape(train_data))
print(np.shape(test_data))

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
def generator():
    #inshape = (64, 1000, 64)
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(1000,64), strides = 2))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu', strides = 2))
        model.add(tf.keras.layers.BatchNormalization())
       # model.add(tf.keras.layers.Conv1D(824, kernel_size=3, activation='relu', strides = 2))
        #model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(512, kernel_size=3, activation='relu', strides = 2))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(256, kernel_size=3, activation='relu', strides = 2))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', strides = 2))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
        model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
        model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
        model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
        #model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))
        #model.add(tf.keras.layers.AveragePooling1D(pool_size = 2))

        model.add(tf.keras.layers.Reshape((1, 1, 64)))

        model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2DTranspose(512, (4, 4), strides=(1, 1), activation='relu')))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding = 'same', activation='relu')))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding = 'same', activation='relu')))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding = 'same', activation='relu')))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding = 'same', activation='relu')))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding = 'same', activation='tanh')))
        #model.add(tf.keras.layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding = 'same', activation='relu'))  
        return model

In [ ]:
generator = generator()
generator.summary()

In [ ]:
def discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(16, (4, 4), strides=(1, 1), input_shape=[128, 128, 3]))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2))))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2))))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2))))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2D(128, (3, 3), strides=(2, 2))))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tfa.layers.SpectralNormalization(tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2))))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))

    return model

In [ ]:
discriminator = discriminator()
discriminator.summary()

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def D_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.fill(np.shape(real_output), 0.9), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
  
def G_loss(fake_output):
    loss =  cross_entropy(tf.ones_like(fake_output), fake_output)
    return loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0004)

In [ ]:
@tf.function
def train_step(images1, images2):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(images1, training=True)        
        real_output = discriminator((images2+0.01*tf.random.uniform(shape=[images2.shape[0],128,128,3], minval=-1, maxval=1, dtype=tf.float64)), training=True)
        fake_output = discriminator((generated_images+0.01*tf.random.uniform(shape=[generated_images.shape[0],128,128,3], minval=-1, maxval=1, dtype=tf.float32)), training=True)

        gen_loss = G_loss(fake_output)
        disc_loss = D_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        
        return gen_loss, disc_loss

In [ ]:
def sv_img(model, epoch, test_input):
    img_list = []
    img = np.load(test_input[random.randrange(0, 29871)])
    img = cv2.resize(img, dsize=(64, 1000))
    img = (img-127.5)/127.5
    img_list.append(img)
    img_array = np.array(img_list)
    prediction = model(img_array, training=False)
    prediction = (prediction[0]*127.5)+127.5
    prediction = prediction.numpy()
    plt.imshow(prediction.astype('uint8'))
    #plt.savefig('/data/raghav/generated_images/generated_img{}.png'.format(epoch), bbox_inches='tight')
    plt.show()

In [ ]:
gen_loss_final = []
disc_loss_final = []
loss = []
epochs = 10
for epoch in range(epochs):
    start = time.time()
    for i in range(0, 1867):
        data1 = []
        data2 = []
        count = 0
        for img in train_data[64*i:]:
            if (count < 64):
                data1 = list(data1)
                data2 = list(data2)
                lms = np.load(img)
                lms = cv2.resize(lms, dsize=(64, 1000))
                lms = (lms-127.5)/127.5
                data1.append(lms)
                data1 = np.array(data1)
                data2.append(cv2.imread(faces_data[img])[:,:,::-1])
                data2 = np.array(data2)
                data2 = (data2-127.5)/127.5
                count = count + 1
            else:
                break
        loss.append(train_step(data1, data2))
    print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    gen_loss_final.append(sum(loss[0]))
    disc_loss_final.append(sum(loss[1]))
    loss = []
    sv_img(generator, epoch + 1, test_data)

In [ ]:
plt.plot(gen_loss_final, label = 'Generator loss')
plt.plot(disc_loss_final, label = 'Discriminator loss')
plt.legend()
plt.show()